#                                   BDS Assignment_VINISHA_D18042

## Q. 1.	Write a Pyspark code for decision tree like exercise you have completed in pyhton. 

In [1]:
#importing libraries and creating connections
from pyspark import SparkContext
sc = SparkContext.getOrCreate()

In [2]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *
sqlContext = SQLContext(sc)

In [3]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Python Spark SQL basic example").config("spark.some.config.option", "some-value").getOrCreate()

In [5]:
#read csv
df = spark.read.csv("iris.csv",header='true',inferSchema=True)
# Displays the content of the DataFrame to stdout
df.show()

+---+------------+-----------+------------+-----------+-------+
|_c0|Sepal_Length|Sepal_Width|Petal_Length|Petal_Width|Species|
+---+------------+-----------+------------+-----------+-------+
|  1|         5.1|        3.5|         1.4|        0.2| setosa|
|  2|         4.9|        3.0|         1.4|        0.2| setosa|
|  3|         4.7|        3.2|         1.3|        0.2| setosa|
|  4|         4.6|        3.1|         1.5|        0.2| setosa|
|  5|         5.0|        3.6|         1.4|        0.2| setosa|
|  6|         5.4|        3.9|         1.7|        0.4| setosa|
|  7|         4.6|        3.4|         1.4|        0.3| setosa|
|  8|         5.0|        3.4|         1.5|        0.2| setosa|
|  9|         4.4|        2.9|         1.4|        0.2| setosa|
| 10|         4.9|        3.1|         1.5|        0.1| setosa|
| 11|         5.4|        3.7|         1.5|        0.2| setosa|
| 12|         4.8|        3.4|         1.6|        0.2| setosa|
| 13|         4.8|        3.0|         1

In [6]:
df

DataFrame[_c0: int, Sepal_Length: double, Sepal_Width: double, Petal_Length: double, Petal_Width: double, Species: string]

In [7]:
#checking the schema of iris data frame
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Sepal_Length: double (nullable = true)
 |-- Sepal_Width: double (nullable = true)
 |-- Petal_Length: double (nullable = true)
 |-- Petal_Width: double (nullable = true)
 |-- Species: string (nullable = true)



In [8]:
#1st row of the dataframe
df.head(1)

[Row(_c0=1, Sepal_Length=5.1, Sepal_Width=3.5, Petal_Length=1.4, Petal_Width=0.2, Species='setosa')]

In [9]:
#importing vector assembler
from pyspark.ml.feature import VectorAssembler

In [10]:
#columns of data frame
df.columns

['_c0',
 'Sepal_Length',
 'Sepal_Width',
 'Petal_Length',
 'Petal_Width',
 'Species']

In [11]:
#separating features and labels
assembler=VectorAssembler(inputCols=['Sepal_Length', 'Sepal_Width', 'Petal_Length', 'Petal_Width'],outputCol='features')

In [12]:
#transforming dataframe using assembler
output=assembler.transform(df)

In [13]:
#StringIndexer encodes a string column of labels to a column of label indices. 
indexer=StringIndexer(inputCol='Species',outputCol='SpeciesIndex')

In [14]:
output_fixed=indexer.fit(output).transform(output)

In [15]:
#transformed schema of dataframe
output_fixed.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Sepal_Length: double (nullable = true)
 |-- Sepal_Width: double (nullable = true)
 |-- Petal_Length: double (nullable = true)
 |-- Petal_Width: double (nullable = true)
 |-- Species: string (nullable = true)
 |-- features: vector (nullable = true)
 |-- SpeciesIndex: double (nullable = true)



In [16]:

final_data=output_fixed.select('features','SpeciesIndex')

In [43]:
#train and test split
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [44]:
#decision tree model
dtc=DecisionTreeClassifier(labelCol='SpeciesIndex',featuresCol='features')

In [45]:
#fitting decision tree to the train data
dtc_model=dtc.fit(train_data)

In [46]:
#prediction on test data
dtc_preds=dtc_model.transform(test_data)

In [47]:
#evaluating model using multi class classification evaluator
my_eval=MulticlassClassificationEvaluator(labelCol='SpeciesIndex')

In [48]:
#F1 score of model
print(my_eval.evaluate(dtc_preds))

0.9106449106449106


In [49]:
#schema of predition
dtc_preds.printSchema()

root
 |-- features: vector (nullable = true)
 |-- SpeciesIndex: double (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = true)



In [50]:
print(dtc_preds)

DataFrame[features: vector, SpeciesIndex: double, rawPrediction: vector, probability: vector, prediction: double]


In [51]:
#Evaluate accuracy
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction",labelCol="SpeciesIndex",metricName="accuracy")
acc=evaluator.evaluate(dtc_preds)      
acc

0.9090909090909091

In [52]:
#Draw a confusion matrix
dtc_preds.groupBy("SpeciesIndex","prediction").count().show()

+------------+----------+-----+
|SpeciesIndex|prediction|count|
+------------+----------+-----+
|         2.0|       0.0|    2|
|         1.0|       1.0|   12|
|         2.0|       2.0|   12|
|         1.0|       0.0|    2|
|         0.0|       0.0|   16|
+------------+----------+-----+



 ## Q.2 Create array of names and publish name and length of each name.

In [27]:
array = sc.parallelize(["Python", "R", "Java",  "Pyspark", "Scala" , "Ruby",'Html','Sql'])
y=array.map(lambda y:(y,len(y)))

y.collect()

[('Python', 6),
 ('R', 1),
 ('Java', 4),
 ('Pyspark', 7),
 ('Scala', 5),
 ('Ruby', 4),
 ('Html', 4),
 ('Sql', 3)]

## Q.3 .	Create array of names and find max, min, standard deviation, variance of length of each element in array.

In [28]:
array = sc.parallelize(["Python", "R", "Java",  "Pyspark", "Scala" , "Ruby",'Html','Sql'])
y=  array.map(lambda y:(len(y)))
y.stats()

(count: 8, mean: 4.25, stdev: 1.713913650100261, max: 7.0, min: 1.0)

## Q.4  Use data set “ocp_data.tsv” attached in sheet (data is delimited with tab space)

In [29]:
from pyspark.sql import Row

In [30]:
lines = sc.textFile("OCP_data.tsv")
parts = lines.map(lambda l: l.split("\t"))


In [31]:
parts.take(5)

[['YEAR', 'GoogleKnowlege_Occupation', 'Show', 'Group', 'Raw_Guest_List'],
 ['1999', 'actor', '01-11-1999', 'Acting', 'Michael J. Fox'],
 ['1999', 'Comedian', '01-12-1999', 'Comedy', 'Sandra Bernhard'],
 ['1999', 'television actress', '01-13-1999', 'Acting', 'Tracey Ullman'],
 ['1999', 'film actress', '01-14-1999', 'Acting', 'Gillian Anderson']]

### 4.(A) 	List no. of shows per year

In [39]:
lines = sc.textFile("OCP_data.tsv")
parts = lines.map(lambda l: l.split("\t"))
show = parts.map(lambda x: (x[0], 1)).reduceByKey(lambda x,y: x+y)
show.top(show.count())


[('YEAR', 1),
 ('2015', 100),
 ('2014', 163),
 ('2013', 166),
 ('2012', 164),
 ('2011', 163),
 ('2010', 165),
 ('2009', 163),
 ('2008', 164),
 ('2007', 141),
 ('2006', 161),
 ('2005', 162),
 ('2004', 164),
 ('2003', 166),
 ('2002', 159),
 ('2001', 157),
 ('2000', 169),
 ('1999', 166)]

### 4.(B)  List no. of shows per year for every occupation

In [33]:
lines = sc.textFile("OCP_data.tsv")
parts = lines.map(lambda l: l.split("\t"))
showperocc = parts.map(lambda x: (x[0]+' ' +x[1], 1))
showperocc.reduceByKey(lambda x,y: x+y).collect()

[('YEAR GoogleKnowlege_Occupation', 1),
 ('1999 actor', 53),
 ('1999 Comedian', 5),
 ('1999 television actress', 5),
 ('1999 Singer-lyricist', 1),
 ('1999 actress', 42),
 ('1999 Singer-songwriter', 2),
 ('1999 Comic', 1),
 ('1999 rock band', 2),
 ('1999 musician', 2),
 ('1999 Film actor', 2),
 ('1999 Model', 1),
 ('1999 singer-songwriter', 2),
 ('1999 film actor', 2),
 ('1999 Vocalist', 2),
 ('1999 writer', 1),
 ('1999 singer', 4),
 ('1999 director', 1),
 ('1999 rapper', 1),
 ('2000 actor', 61),
 ('2000 actress', 29),
 ('2000 model', 2),
 ('2000 Singer-songwriter', 2),
 ('2000 journalist', 1),
 ('2000 activist', 1),
 ('2000 Media person', 1),
 ('2000 NA', 4),
 ('2000 former us senator', 3),
 ('2000 Journalist', 2),
 ('2000 film actress', 3),
 ('2000 stand-up comedian', 1),
 ('2000 Reporter', 1),
 ('2000 writer', 4),
 ('2000 professional wrestler', 1),
 ('2000 reality show contestant', 1),
 ('2000 television host', 1),
 ('2000 game show host', 1),
 ('2000 Guitarist', 1),
 ('2000 magicia

### 4.(C)  Find top occupation for every year.

In [34]:
from collections import Counter

In [36]:
occ=parts.map(lambda x: (x[0],x[1]))
occ1 = occ.groupByKey().mapValues(list)
occ2=occ1.map(lambda x: (x[0],Counter(x[1]).most_common(1)))
occ2.sortByKey().collect()

[('1999', [('actor', 53)]),
 ('2000', [('actor', 61)]),
 ('2001', [('actor', 62)]),
 ('2002', [('actor', 64)]),
 ('2003', [('actor', 47)]),
 ('2004', [('actor', 31)]),
 ('2005', [('actor', 25)]),
 ('2006', [('actor', 33)]),
 ('2007', [('actor', 15)]),
 ('2008', [('actor', 20)]),
 ('2009', [('actor', 19)]),
 ('2010', [('actor', 30)]),
 ('2011', [('actor', 29)]),
 ('2012', [('actor', 20)]),
 ('2013', [('actor', 37)]),
 ('2014', [('actor', 31)]),
 ('2015', [('actor', 19)]),
 ('YEAR', [('GoogleKnowlege_Occupation', 1)])]

### 4.(D) 	Which guest have attended maximum shows

In [40]:
lines = sc.textFile("OCP_data.tsv")
parts = lines.map(lambda l: l.split("\t"))
guest = parts.map(lambda x: (x[4], 1)).reduceByKey(lambda x,y: x+y)
g=guest.top(guest.count())
#print(type(g[1]))
sorted(g,key=lambda x: x[1], reverse=True)[0]

('Fareed Zakaria', 19)

### 4.(E) 	List occupation under each group 

In [41]:
lines = sc.textFile("OCP_data.tsv")
parts = lines.map(lambda l: l.split("\t"))
occp = parts.map(lambda x: (x[3]+',' +x[1], 1))
occp.reduceByKey(lambda x,y: x+y).collect()

[('Comedy,Comedian', 39),
 ('Musician,Singer-lyricist', 2),
 ('Media,model', 6),
 ('Media,Model', 3),
 ('Politician,us senator', 50),
 ('Musician,pianist', 1),
 ('Media,writer', 30),
 ('Media,television host', 39),
 ('Acting,televison actor', 1),
 ('Media,film director', 14),
 ('Acting,Film actress', 9),
 ('Business,businesswoman', 1),
 ('Advocacy,activist', 14),
 ('Media,Media person', 1),
 ('Media,radio personality', 3),
 ('Media,commentator', 20),
 ('Media,Journalist', 72),
 ('Politician,former senator from kansas', 1),
 ('Musician,Singer', 9),
 ('Athletics,professional wrestler', 1),
 ('Political Aide,former press secretary', 1),
 ('Comedy,comic', 1),
 ('Media,game show host', 1),
 ('Musician,Guitarist', 2),
 ('Musician,Musician', 5),
 ('Politician,Former United States Secretary of Labor', 6),
 ('Athletics,boxer', 1),
 ('Musician,Band', 2),
 ('Misc,chef', 3),
 ('Academic,professor', 22),
 ('Media,television presenter', 3),
 ('Misc,Lawyer', 5),
 ('Political Aide,white house sommunic

### 4.(F) 	How many show per group of year and occupation

In [42]:
lines = sc.textFile("OCP_data.tsv")
parts = lines.map(lambda l: l.split("\t"))
showpergroupofyearandoccupation = parts.map(lambda x: (x[0]+' ' +x[3]+' : '+x[1], 1))
showpergroupofyearandoccupation.reduceByKey(lambda x,y: x+y).collect()

[('1999 Acting : actor', 53),
 ('1999 Acting : television actress', 5),
 ('1999 Acting : film actress', 3),
 ('1999 Acting : actress', 42),
 ('1999 Musician : Singer-songwriter', 2),
 ('1999 Musician : rock band', 2),
 ('1999 Musician : musician', 2),
 ('1999 Acting : Film actor', 2),
 ('1999 Politician : us senator', 2),
 ('1999 Comedy : Stand-up comedian', 4),
 ('1999 Musician : singer', 4),
 ('1999 Media : television host', 1),
 ('1999 Media : muppet', 1),
 ('1999 Media : film director', 1),
 ('1999 Musician : rapper', 1),
 ('2000 Athletics : football player', 1),
 ('2000 Acting : Film actress', 2),
 ('2000 Musician : singer-songwriter', 3),
 ('2000 Comedy : comedian', 8),
 ('2000 Business : businesswoman', 1),
 ('2000 Musician : singer', 1),
 ('2000 Advocacy : activist', 1),
 ('2000 Media : Media person', 1),
 ('2000 Acting : film actor', 2),
 ('2000 Politician : former us senator', 3),
 ('2000 Politician : american politician', 5),
 ('2000 Media : radio personality', 1),
 ('2000 M